In [ ]:
%%time

import scipy.integrate as integrate
from scipy.integrate import solve_ivp
import numpy as np

# This controls which part of the assignment we will be solving for
mode = 'galaxy'

if mode == 'solar':
    print('Solar check:')
    G = 6.6e-11 # N m^2/kg^2

    ### The following section contains the initial postions, velocities and masses for the four bodies.
    
    sun_r0 = np.array([0.0, 0.0, 0.0]) # m
    sun_v0 = np.array([0.0, 0.0, 0.0]) # m/s
    sun_M = 2e30 # kg
    
    earth_r0 = np.array([149.6e9, 0.0, 0.0]) # m
    earth_v0 = np.array([0.0, 29.8e3, 0.0]) # m/s
    earth_M = 5.97e24 # kg
    
    venus_r0 = np.array([108e9, 0.0, 0.0])
    venus_v0 = np.array([0.0, 35e3, 0.0])
    venus_M = 5e24
    
    mars_r0 = np.array([230e9, 0.0, 0.0])
    mars_v0 = np.array([0.0, 24e3, 0.0])
    mars_M = 6.4e23

    # organising arrays and functions to use for dydt function
    y0 = np.array([ [sun_r0, venus_r0, earth_r0, mars_r0],
                 [sun_v0, venus_v0, earth_v0, mars_v0 ] ])
    print(f'y0 shape: {y0.shape}')
    m = np.array([sun_M,venus_M,earth_M,mars_M])
    print(f'm: {m}')
    a = 1e-20                                # A regularisation term for the force calculation
       
    def dydt(t,y_flat): 
      # reshape y_flat to original shape, dependent on number of bodies
      y_shape = y_flat.shape[0]//(2*3)      # find number of bodies via division of 6: 2 arrays, of distance and velocity & 3D coords (x,y,z) indicies
      y = y_flat.reshape(2,y_shape,3)       # reshape y for easier extraction of vectors by indexing ([radius, velocity], # of bodies, 
      
      r = y[0,:]                               # select all position vectors in first row
      v = y[1,:]                               # select all velocity vectors in second row        
            
      drdt = v

      R = r[:,None] - r 
      F = G*m[None,:,None]*R/np.power(np.linalg.norm(R + a, axis = 2)[:,:,None], 3)    # force Fij, where the j/=i has not been considered. Rectified with small value 'a'
      dvdt = np.sum(F, axis = 1)    # sum all vector forces along coloumns
      
        # {Sun, Venus, Earth, Mars} = {0,1,2,3} as indicies
        
      drdt = drdt.flatten()
      dvdt = dvdt.flatten()
      return np.array([drdt,dvdt]).flatten()

    dy0dt = dydt(0,y0.flatten()).reshape(2,4,3)
    print(f'd/dt v0: {dy0dt[1,:,:]}')

    # check if this matches the expected values of dv0/dt

    # The end point of simulation, this is 10 years
    t_max = 31557600.0*10 # s
    t_span = (0,t_max)

    # How far out to plot our visualisation later
    vis_radius = 300e9 # m
    print()
    
elif mode == 'galaxy':
    print('Galaxy Check')
    G = 4.49e-18 # kpc^3/M_solar/kyear^2

    # Some constants for generating the locations of the bodies
    R_gen = 50 # kpc
    z_scale_factor = 0.25
    N_bodies = 200
    M_bodies = 1e8 # M_solar
    rc = 1 # kpc
    a = rc/np.power((2**(2/5) - 1),1/2)
    M = M_bodies*np.random.rand(N_bodies)
    
    # create random set of initial positions & velocities within the sphere, R_gen
    
    def rand_sphere(number_of_bodies):
    
        rand = np.random.rand(number_of_bodies,3)
        phi = 2*np.pi*rand[:,0]
        theta = np.arccos(1-2*rand[:,1])
        #r = R_gen*rand[:,2]**(1/3)
        r = R_gen*0.25*rand[:,2]    # for a flatter galaxy with denser centers

        r0 = r*np.array([ np.sin(theta)*np.cos(phi),    # spherical random distributions of initial positions
                  np.sin(theta)*np.sin(phi),
                  np.cos(theta) ])
        r0 = r0.T
        
        print(f'r0 initialised, with shape: {r0.shape}')
        return r0

    # setting initial conditions
    
    r0 = rand_sphere(N_bodies)
    print(f'r0: {r0[0:4,:]}')    # check first 4 radii
    v0 = np.zeros((N_bodies,3))
    y0 = np.array([ r0,v0 ])     # expect indicies ([radius, velocity], # of bodies, [x,y,z])
    print(f'y0 shape: {y0.shape}')
        
    def dydt(t,y_flat): # for galaxy to account for Plummer Radius & radial acceleration
      # reshape y_flat to original shape, dependent on number of bodies
      bodies = y_flat.shape[0]//(2*3)       # 2 components, of distance and velocity & 3 coordinates (x,y,z)  
      y_shape = y_flat.shape[0]//(2*bodies)
      y = y_flat.reshape(2,bodies,y_shape)          # reshape y for easier extraction of vectors 
      
      r = y[0,:]                               # select all position vectors in first row
      v = y[1,:]                               # select all velocity vectors in second row        
            
      drdt = v
      R = r[:,None] - r 
      F = G*M[None,:,None]*R/np.power(np.linalg.norm(R + a, axis = 2)[:,:,None], 3)
      dvdt = np.sum(F + v**2/r, axis = 1)    # adding radial acceleration
      
      drdt = drdt.flatten()
      dvdt = dvdt.flatten()
      return np.array([drdt,dvdt]).flatten()

    # obtain acceleration
    dy0dt = dydt(0,y0.flatten()).reshape(2,N_bodies,3)
    print(f'dy0dt shape: {dy0dt.shape}')
    dvdt = dy0dt[0,:,:]
    drdt = dy0dt[1:,:]

    print(f'dvdt for t = 0: {dvdt[0:4,:]}')

    # create the initial vector variables from initial acceleration
    v_x = np.random.rand(N_bodies)
    v_y = -dvdt[:,0]/(dvdt[:,1] + 1e-20)*v_x    # v_y = -a_x/a_y*v_x
    v0 = np.array([v_x,v_y,np.zeros(N_bodies)]).T
    y0 = np.array([ r0,v0 ])
    print(f'v0: {y0[1,0:4,:]}')
        
    # The end point of the simulation, this is 1 billion years
    t_max = 1000000 # kyr
    t_span = (0,t_max)

    # How far out the visualisation should go
    vis_radius = 50 # kpc
    print()

else:
  print("Invalid mode")

# number of points for solution to be evaluated over
points = 50
t_grid = np.linspace(0,t_max,points)
solution_y_t = solve_ivp(dydt, t_span, y0.flatten(), method = 'RK45', t_eval=t_grid)   #shape of solveivp = (n,n_points).T

def y_reshape(y_flat):
    # reshape y_flat by reorienting indicies from (y_flat, t_grid) -> (t_grid, y_flat)
    y = y_flat.T
    y_shape = y[0].shape[0]//(2*3)    # find number of bodies via division of 6: 2 arrays, of distance and velocity & 3D coords (x,y,z) indicies
    y = y.reshape(y.shape[0],2,y_shape,3)    # obtain solution in the shape of (t_grid,[radius, velocity], # of bodies, [x,y,z])
    r = y[:,0,:,:]
    v = y[:,1,:,:]
    print('y_reshape success')    # check if code ran thoroughly
    return r,v

r,v = y_reshape(solution_y_t.y)

if (np.any(r) < vis_radius):
    print('Radii exceeds boundaries, check dydt')
else:
    print(f'Radii within boundaries, with shape {r.shape}')

print(f'r(t) shape: {r.shape}')
print(f'r(t=0) == r0: {np.all(r[0,0:4,:] == y0[0,0:4,:])}') # check if all values match with initial conditions

Galaxy Check
r0 initialised, with shape: (200, 3)
r0: [[-0.28127685  1.54384297  0.55041425]
 [-2.88817636  0.96043605 -3.83800491]
 [ 0.47820592 -0.34383335 -0.29174387]
 [-0.38484857 -7.19560518  8.49382618]]
y0 shape: (2, 200, 3)
dy0dt shape: (2, 200, 3)
dvdt: [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
v0: [[ 0.79226622 -0.          0.        ]
 [ 0.53816596 -0.          0.        ]
 [ 0.8905251  -0.          0.        ]
 [ 0.37446677 -0.          0.        ]]



In [2]:
"""Functions to provide the plot kwargs in the two cases."""
from __future__ import annotations
def solar_plot_params(body_names: list[str], size_factor: float = 1) -> dict[str, list]:
    """
    Get the color and size key-word argument arrays for the solar system.

    The body names provides the order of the bodies in the plot to ensure proper matching

    Parameters
    ----------
    body_names : list[str]
        Order of bodies as simulated
    size_factor : float, optional
        Multiplicative factor to adjust size (area) of all bodies in plot, by default 1

    Returns
    -------
    dict[str, list]
        Unpack output from this function with ** into ax.scatter.
    """
    # Note sizes are area, not radius
    solar_sizes = {
        "sun": 50,
        "earth": 28,
        "venus": 24,
        "mars": 20,
    }
    solar_colours = {
        "sun": "yellow",
        "earth": "blue",
        "venus": "orange",
        "mars": "red",
    }
    res = {"s":[],"c":[]}
    for body in body_names:
        body = body.lower()
        res["s"].append(solar_sizes[body] * size_factor)
        res["c"].append(solar_colours[body])
    return res

def galaxy_plot_params(n_bodies: int, cmap: str = "hsv", map_range: tuple[float] = (0,1), size: int = 7):
    """
    Get the colour, size and cmap key-word argument values for the galaxy simulation.

    Map range allows the accessing of a subset of a colormap through a range narrower than (0,1).
    See https://matplotlib.org/stable/users/explain/colors/colormaps.html for more colormaps.

    Parameters
    ----------
    n_bodies : int
        Number of bodies in galaxy.
    cmap : str, optional
        Color map to use, by default "hsv"
    map_range : tuple[float], optional
        Subset of colormap to randomly sample, by default (0,1)
    size : int, optional
        Area of bodies, by default 7

    Returns
    -------
    _type_
        _description_
    """
    res = {
        "s": size,
        "c": np.random.uniform(*map_range, size=n_bodies), 
        "cmap": cmap,
    }
    return res

In [3]:
import matplotlib.animation
import matplotlib.pyplot as plt
import matplotlib as mpl

def get_body_coordinates(t):
    """
    Returns the coordinates of each body in a tuple. If we are simulating a solar system then 
    only the x and y coordinates are returned.

    Parameters
    ----------
    t : int
        The time step for which the body coordinates should be returned, where 0 is the first time step.
    """
    #########################
    # Fill in the dots below with the output of your simulation
    # Take your 'solution_y_t', index it with the 't' argument of this function, 
    # and use it to get the x, y, and z coordinates of the simulation bodies.
    #########################

    #r,v = y_reshape(solution_y_t.y)
  
    body_x_coordinates = r[t,:,0]
    body_y_coordinates = r[t,:,1]

    if mode == 'solar':
        return body_x_coordinates, body_y_coordinates, None
    elif mode == 'galaxy':
        body_z_coordinates = r[t,:,2]
        return body_x_coordinates, body_y_coordinates, body_z_coordinates

plt.rcParams["animation.html"] = "jshtml" # Makes animations work better in a notebook
plt.rcParams['figure.dpi'] = 100 # Sets the size of the animation in dots per inch
plt.ioff()

plt.close() # Ensure we get a fresh canvas
fig = plt.figure() # Create a new figure to animate in

# First we initiate the plot using the first time slice
body_init_x_coordinates, body_init_y_coordinates, body_init_z_coordinates = get_body_coordinates(0)

print(f'Initial coords (x,y,z) = {body_init_x_coordinates[0:4], body_init_y_coordinates[0:4]}') # body_init_z_coordinates[0:4]}')

# Actually plotting the data, note that we collect the output from scatter
if mode == 'solar':
    ax = plt.axes() # standard 2D axes for the solar system
    # solar_plot_params contains colour and size arguments
    plot_kwargs = solar_plot_params(["sun","venus","earth","mars"], size_factor=2)
    # 2D scatter plot of the x and y coordinates of the solar system bodies
    bodies = ax.scatter(body_init_x_coordinates, body_init_y_coordinates, **plot_kwargs)
elif mode == 'galaxy':
    ax = plt.axes(projection='3d') # 3D axes for the galaxy simulation
    # Size and colour again
    plot_kwargs = galaxy_plot_params(N_bodies,cmap="spring",map_range=(0.6,0.7), size=12)
    # Similar to the solar system case, but this 3D scatter plot has three inputs: x, y, and z
    bodies = ax.scatter(body_init_x_coordinates, body_init_y_coordinates, body_init_z_coordinates, **plot_kwargs)

# Finish initialisation by setting the bounds and labels
ax.set_xlim(-vis_radius,vis_radius)
ax.set_xlabel("x")
ax.set_ylim(-vis_radius,vis_radius)
ax.set_ylabel("y")
if mode == 'galaxy':
    ax.set_zlim(-vis_radius,vis_radius)
    ax.set_zlabel("z")
ax.set_aspect('equal') # Ensure we get a square plot

def animate(t):
    """
    Animation function, at any given timestep returns the updated 'artists' bodies so that the animator
    can update the plot.
    """
    body_x_coordinates, body_y_coordinates, body_z_coordinates = get_body_coordinates(t)

    # Now we update the coordinates. They are called offsets underneath for a scatter plot
    if mode == 'solar':
        # Set offsets requires an (N, 2) array
        bodies.set_offsets(np.array([body_x_coordinates, body_y_coordinates]).T)
    elif mode == 'galaxy':
        # No setter method exists for 3d so we set the points manually        
        bodies._offsets3d = (body_x_coordinates, body_y_coordinates, body_z_coordinates)

    # We also spin the plot inside
    if mode == 'galaxy':
        # We only set the z limits and camera rotation for a 3D plot
        ax.azim = ax.azim + 0.2
    
    return bodies, # Must return a tuple or list of the 'artists', the comma ensures this is a tuple

# By passing blit=True, we are telling FuncAnimation that our animate function will return the artists with
# the changed data. As a result it automatically only updates the relevant parts of the plot.
anim = matplotlib.animation.FuncAnimation(fig = fig, func = animate, frames=(points-1), interval = 100, blit=True)
plt.close()
anim

NameError: name 'r' is not defined

<Figure size 640x480 with 0 Axes>

In [ ]:
# Energy function

M = m[:,None] * m
np.fill_diagonal(M, 0)                      # 0 diagonal, since masses cannot influence itself on potential

print(M)

def sys_energy(y_flat):
       
    u = np.zeros((r.shape[1],r.shape[1]))
    U = np.zeros(r.shape[0])
    for t in range(0,len(t_grid)-1):
        for i in range(0,len(m)-1):
            for j in range(0,len(m)-1):
                u[i,j] = -G*M[i,j]/(np.linalg.norm( r[t,i,:] - r[t,j,:] + a ) )    # given potential energy form
                #K = 1/2*m[i]*np.linalg.norm(v[t,i,:])**2                           # kinetic energy of system
        U[t] = np.sum(1/2*u)

    total = U #+ K
    return total

plt.plot(t_grid,sys_energy(solution_y_t.y))
plt.ylabel('Energy, U')
plt.xlabel('s')
plt.title('Potential Energies of the Bodies')
plt.show()

In [ ]:
## Troubleshooting & create function to transform solution to objects r & v of size (4,3,t_grid)

print(f'Solution for t = 0: {solution_y_t.y[:,0]}')
print()
sol_y = solution_y_t.y.T
print(f'Solution transposted for t = 0 {sol_y[0,:]}') # transposed such that the indicies [solutions, time] -> [time, solutions] to reshape the solution elements
print()
sol_y = sol_y.reshape(50,2,4,3)
print(f'Reshaped: {sol_y[0,:,:,:]}')
print()
r = sol_y[:,0,:,:]
print(f'r for t = 0 {r[0,:,:]}') #r[time,body,(x,y,z)]

In [ ]:
## Troubleshooting & diagnosing how to make dydt via broadcasting

r = y0[0,:]                               # select all position vectors in first row
v = y0[1,:]                               # select all velocity vectors in second row        

M = m[:,None] * m
R = r[:,None] - r 
#print(f'm = {m}')
#print()
#print(f'M = {M}')
#print()
#print(f'R = {R}')
#print()
F_mj = m[None,:,None]*R # correct orientation
F_sun = m[:,None]*R[0,:,:]
F_sun_earth = m[2]*R[0,2,:]    # m[2] = earth, radii vector R[sun,earth,(x,y,z)]
#print(f'Forces on Sun {F_sun}')
#print()
print(f'Forces on Sun & Earth {F_sun_earth}')
print()
print(f'm[None,:,None] ={F_mj[0,:,:]}') # gives correct forces of bodies acting on the sun, given by F_sun_earth which is expected to be the third row of sun's array
print()

## check distance calculations
print(f'Magnitude between Sun & Earth: {np.linalg.norm(R[0,2,:])}')
print()
print(f'Magnitude of R, vectors are rows: {np.linalg.norm(R, axis = 2)}')
print()
print(f'Unit vector, r/|r| of Sun against Earth: {R[0,2,:]/np.linalg.norm(R, axis = 2)[0,2]}')
print()
print(f'Unit vector, r/|r| of all bodies against the sun: {(R/np.linalg.norm(R + a, axis = 2)[:,:,None])[0,:,:]}')
print()
F = G*F_mj/(np.power(np.linalg.norm(R + a, axis = 2)[:,:,None],3))
print(f'forces, sun: {F[0,:,:]}') # this is the correct form
print()
print(f'dv/dt, sun: {np.sum(F, axis = 1)[0,:]}')
print(F[0,0,:]+F[0,1,:]+F[0,2,:]+F[0,3,:]) # this must agree with previous print statement for correct summation

# by logic, this should be the correct form via matching expected with broadcasted results. Why the planets shoot off to a new system idk

F_sun_vectors = G*np.array([ m[0]*R[0,0,:]/np.power(np.linalg.norm(R[0,0,:] + a), 3),
                             m[1]*R[0,1,:]/np.power(np.linalg.norm(R[0,1,:] + a), 3),
                             m[2]*R[0,2,:]/np.power(np.linalg.norm(R[0,2,:] + a), 3),
                             m[3]*R[0,3,:]/np.power(np.linalg.norm(R[0,3,:] + a), 3)])
print(f'Forces acting on the sun: {F_sun_vectors}')
dvdt_sun = np.sum(F_sun_vectors, axis = 0)
print(f'Forces summed: {dvdt_sun}')